In [1]:
import websocket
import json
import time
from binance import Client
import lib.DB_lib as dbl
import Constants

In [4]:
columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_taker_ast_vol', 'no_of_trades'
        , 'buy_base_ast_vol', 'taker_buy_quote_ast_vol','ignore']
api_key = 'HGuZ0EOwG2CgAAq7CLx7JjLCTNHd2NL5BzqBAfa8qiuQkcVBaUySB'
api_secret = 'QGaXTPSjsROXj24O7y5AxYH1CjAWNFqssmmHOEb5HCAzoIfY3gEs3ZhSGgfDg2C6'
symbol = Constants.symbol
interval = Constants.interval
crypto_interval = ''
if interval == '5m':
    crypto_interval = Client.KLINE_INTERVAL_5MINUTE
elif interval == '15m':
    crypto_interval = Client.KLINE_INTERVAL_15MINUTE
elif interval == '30m':
    crypto_interval = Client.KLINE_INTERVAL_30MINUTE
elif interval == '1h':
    crypto_interval = Client.KLINE_INTERVAL_1HOUR
elif interval == '4h':
    crypto_interval = Client.KLINE_INTERVAL_4HOUR    

table_name = dbl.get_raw_table(symbol,interval)
l = []

def get_historical_data(asset,start_time, end_time):
    client = Client(api_key, api_secret)
    klines = client.get_historical_klines(asset, Client.KLINE_INTERVAL_5MINUTE,start_time,end_time)
    return klines

def on_message(ws, message):
    try:
        data = json.loads(message)
        if data['k']['x'] == True:
            l.append(data)
            dbl.insert_table_stream(table_name,data)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")    

def on_error(ws, error):
    print(error)
    
def on_close(close_message):
    print(close_message)
    
def streamKline(symbol, interval):
    socket = f'wss://stream.binance.com:9443/ws/{symbol}@kline_{interval}'
    ws = websocket.WebSocketApp(socket,
                                on_message=on_message,
                                on_error=on_error,
                                on_close=on_close)
    ws.run_forever()
    


def get_crypto_data(symbol,interval):
     last_time_stamp = dbl.get_last_timestamp(symbol,interval)
     current_time_stamp = int(time.time() * 1000)
     # last_time_stamp = 1598898600000
     print(last_time_stamp)
     print(current_time_stamp)
     print(symbol)
     print(crypto_interval)
     client = Client(api_key, api_secret)
     klines = client.get_historical_klines(symbol.upper(), crypto_interval,last_time_stamp+1000,current_time_stamp)
     print(len(klines))
     if len(klines)>0:
         dbl.insert_table_historical(table_name,klines[:-1])
     # streamKline(symbol, interval)
     
get_crypto_data(symbol, interval)       

s01_maticusdt_15m
1714746600000
1714790695991
maticusdt
15m
48
